# Momentum-space scattering wavefunctions as eigenstates

---
This code is made available as part of the **FRIB-TA Summer School: A Practical Walk Through Formal Scattering Theory: Connecting Bound States, Resonances, and Scattering States in Exotic Nuclei and Beyond**, held virtually
August 4-6, 2021.

https://fribtascattering.github.io/

*Organizers/Lecturers:*
- Kévin Fossez (MSU/ANL)
- Sebastian König (NCSU)
- Heiko Hergert (MSU)

*Author:* Sebastian König

*Last update:* Aug 11, 2021

---

We solve once more the momentum-space Lippmann-Schwinger equation in a fixed partial wave,

\begin{equation}
T(E_k+\mathrm{i}\varepsilon;p,k) = V(p,k)
+\int\frac{\mathrm{d}q\,q^2}{2\pi^2} V(p,q)
G_0(E_k+\mathrm{i}\varepsilon;q)
T(E_k+\mathrm{i}\varepsilon;q,p) \,,
\end{equation}

with $\varepsilon\to0$ implied.

## System and potential

We consider here again our previously introduced simple system (mass $m=1$ and $\hbar c=1$), and Gaussian potential:

In [ ]:
from lib.system import *
from lib.potential import *

sys = System()
V = V_Gauss(sys, -4.0, 2.0)

## Equation solving

We copy the basic infrastructure for solving the half off-shell Lippmann-Schwinger equation from what we used before:

In [ ]:
from lib.mesh import *
from lib.operator import *

G0 = G_0(sys)

As only modification, we pass here the mesh explicitly as a parameter:

In [ ]:
def kernel(mesh, E):
  factor = 0.5 / np.pi**2

  K = map(
    lambda p: list(map(
      lambda qw: factor * qw[1] * qw[0]**2 \
        * (G0.residue(qw[0]) if qw[2] else G0(E, qw[0])) \
        * V.get(0, p, qw[0]),
      mesh.pws()
    )), mesh.ps()
  )

  return np.asarray(list(K))

In [ ]:
def solve(mesh, k):
  mesh.push_pv(k)
  
  mat = np.identity(mesh.size()) - kernel(mesh, sys.e_from_k(k))
  vec = np.asarray(list(map(lambda p: V.get(0, p, k), mesh.ps())))
  
  sol = np.linalg.solve(mat, vec)
  
  mesh.pop_pv()
  
  return sol;

`solve` then gives us $T(E_k;p,k)$ for fixed $k$ and disretized $p$, and with that half off-shell T-matrix we
and construct scattering wavefunctions as

\begin{equation}
 \psi^{(+)}_k(q)
 = \frac{2\pi^2\delta(q-k)}{k^2} + \frac{2\mu T(E_k;q,k)}{k^2 - q^2 + \mathrm{i}\varepsilon} \,.
\end{equation}

We now want to demonstrate in which sense exactly these wavefunctions define eigenstates of the Hamiltonian, i.e.,
we want to verify that indeed

\begin{equation}
 H|\psi^{(+)}_k\rangle = E_k|\psi^{(+)}_k\rangle
\end{equation}

based on our discretized represenation of $|\psi^{(+)}_k\rangle$.

## Delta functions via interpolation

We note that in overlaps $\langle\phi|\psi^{(+)}_k\rangle$ with smooth $|\phi\rangle$, the first term can be evaluated analytically.  While that is an obvious thing to do, it actually *complicates* the numerical implementation.  What we would like instead is a simple vector representation of the wavefunction as a whole.

To that end, we use an **interpolator class** provided by our library.  In its standard mode of operation, it can be used as an alternative to `interp1d` from `scipy.interpolate` (which we used previously):

In [ ]:
from lib.mesh import *
from lib.interpolation import *

mesh = GaulegMesh(16, 0.0, 4.0)
interp = GlobalSplineInterpolator(mesh.ps())

fs = list(map(lambda p: np.sin(p), mesh.ps()))

print(interp(fs, 1.0))
print(np.sin(1.0))

The way this interpolator works is by evaluating a particular set of "spline functions" $S_i$ at the target point $p_0$, and summing the result with the target function evaluated at the mesh points used to construct the interpolator:

\begin{equation}
 f(p_0) = \sum_i S_i(p_0) f(p_i)
\end{equation}

Note that the right-hand side is a dot product of the two vectors $S_i(p_0)$ and $f(p_i)$, and $S_i(p_0)$ acts like a discretized version of $\delta(p-p_0)$.  We can use this as follows:

In [ ]:
p0 = 1.0
ds = [interp.S(i, p0) for i in range(0, mesh.size())]

print(np.dot(ds, fs))

With this, constructing a discretized delta function for a given integration mesh is simply a matter of putting in the appropriate weights:

In [ ]:
def delta(mesh, p0):
  interp = GlobalSplineInterpolator(mesh.ps())
  return [interp.S(i, p0) / mesh.w(i) for i in range(0, mesh.size())]

## Full wavefunction

We now have all we need to represent the full wavefunction on the mesh as

\begin{equation}
 \psi_i = \psi^{(+)}_k(p_i) \,.
\end{equation}

Of course, we the second term,

\begin{equation}
 \frac{2\mu T(E_k;q,k)}{k^2 - q^2 + \mathrm{i}\varepsilon} = G_0(E_k+\mathrm{i}\varepsilon) T(E_k;q,k)
\end{equation}

we still need to account for the singularity in the limite $\varepsilon\to0$, handled by our principal-value integration.

Our `solve` function already returns the residue of $T(E_k;q,k)$ at $q=k$ as the extra point in our vector, and we need only make sure we multiply it by the appropriate residue of the Green's function:

In [ ]:
import copy

def psi(mesh, k):
  ts = solve(mesh, k)
  
  mesh_pv = copy.deepcopy(mesh)
  mesh_pv.push_pv(k)
  
  ds = delta(mesh_pv, k)
  e = sys.e_from_k(k)
  
  wf = [
    (np.pi**2 / sys.mu * d / k**2 + G0(e, pw[0]) * t if not pw[2] else t * G0.residue(k))
    for pw, d, t in zip(mesh_pv.pws(), ds, ts)
  ]
  
  return wf, mesh_pv

Note that for later convenience we return along with the wavefunction a copy of the mesh that has the principal-value point left.

## Hamiltonian and inner products

Now, in order to calculate $H|\psi^{(+)}_p\rangle$, we of course need a matrix representation of $H = H_0+V$.  The kinetic part $H_0$ is simply a diagonal matrix in momentum space, but for the potential part we need to take into account that its application is defined via an integral, just like when we constructed the kernel matrix for solving the Lippmann-Schwinger equations:

\begin{equation}
 \langle p|V|\psi\rangle = \frac{1}{2\pi^2}\int\mathrm{d}q\,q^2 V(p,q) \psi(q)
\end{equation}

We need to discretize this integral to obtain the Hamiltonian defined on a given mesh:

In [ ]:
def ham(sys, V, mesh):
  factor = 0.5 / np.pi**2
  
  V_mat = np.asarray(list(map(
    lambda pw: list(map(
      lambda qw: factor * qw[1] * qw[0]**2 * V.get(0, pw[0], qw[0]),
      mesh.pws()
    )) if not pw[2] else np.zeros(mesh.size()), mesh.pws()
  )))
  
  H0_mat = np.diag(list(map(
    lambda p: 0.5 * p**2 / sys.mu,
    mesh.ps()
  )))
  
  return H0_mat + V_mat

Since we want to apply the Hamiltonian to scattering states, we have accounted in the above definition for the possibility of principal-value contributions in the mesh.

As a bonus, with the Hamiltonian in matrix form, we can easily look for bound states via direct diagonalization (finding exactly one):

In [ ]:
H = ham(sys, V, mesh)

es, wfs = np.linalg.eig(H)

print(es)

To proceed we note that in the partial-wave projected momentum-space formalism inner products are generally defined as

\begin{equation}
 \langle f|g\rangle = \frac{1}{2\pi^2}\int\mathrm{d}q\,q^2 f(q) g(q)
\end{equation}

and the integral $\int\mathrm{d}q\,q^2$ turns into sum $\sum_j w_j p_j^2$ after discretization.  For bras and kets represeted as simple vectors over a `mesh`, we obtain the following generic implementation:

In [ ]:
def inner(mesh, fs, gs):
  acc = 0
  for qw, f, g in zip(mesh.pws(), fs, gs):
    acc += qw[1] * qw[0]**2 * f * g
    
  return 0.5 * acc / np.pi**2

**Note that operator application in this formalism, as considered above for the potential, is not symmetric!**  They always act to the right, and the integration measure becomes associated with the column index of the operator:

\begin{equation}
 \int\mathrm{d}q\,q^2 V(p,q) \psi(q)
 \to
 \sum_j w_j q_j^2 V_{ij} \psi_j
\end{equation}

for $V_{ij} = V(p_i,p_j)$ and $\psi_j = \psi(p_j)$.
 

## Bound state

Before we move on to scattering states, let's look at the bound state as an example and verify that

\begin{equation}
 E_b = \frac{\langle\psi_b|H|\psi_b\rangle}{\langle\psi_b|\psi_b\rangle} \,.
\end{equation}

In [ ]:
i_b = np.argwhere(es < 0)[0]

e_b = es[i_b]
psi_b = wfs[:,i_b]

norm = inner(mesh, psi_b, psi_b)

print(inner(mesh, psi_b, H @ psi_b) / norm)
print(e_b)

## Scattering state

Taking matrix elements like this (although numerically possible with our fully discretized represenation) does **not** work for scattering states:

In [ ]:
mesh = GaulegMesh(16, 0.0, 4.0)

k = 1.0
psi_k, mesh_k = psi(mesh, k)

H_k = ham(sys, V, mesh_k)

norm = inner(mesh_k, psi_k, psi_k)

print(inner(mesh_k, psi_k, H_k @ psi_k) / norm)
print(sys.e_from_k(k))

Instead, we need to take into account that scattering wavefunctions describe **generalized eigenstates**, satisfying

\begin{equation}
 \langle\phi|H|\psi^{(+)}_k\rangle = E_k\langle\phi|\psi^{(+)}_k\rangle
\end{equation}

for any **test function** $\phi(q)$.

For concreteness, we choose Gaussian wave packets

\begin{equation}
 \phi_{p,\sigma}(q) = \exp\left({-}\frac{(q-p)^2}{2\sigma^2}\right)
\end{equation}

as test functions:

In [ ]:
def wave_packet(mesh, p, sigma=1.0):
  return [np.exp(-(p - q)**2 / (2.0 * sigma**2)) for q in mesh.ps()]

And indeed with these we can verify the generalized eigenstate property:

In [ ]:
p = 2.0
sigma = 1.0
phi = wave_packet(mesh_k, p, sigma=sigma)

H_k = ham(sys, V, mesh_k)

norm = inner(mesh_k, phi, psi_k)

print(inner(mesh_k, phi, H_k @ psi_k) / norm)
print(sys.e_from_k(k))